<a href="https://colab.research.google.com/github/shahd1995913/CNN-Architectures/blob/master/plant2022_part1_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Plant Classification

### [Link of  code ](https://www.kaggle.com/emmarex/plant-disease-detection-using-keras/notebook).

### Data : 2022-2-8 



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
import pickle
import cv2
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf
from os import listdir
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    BatchNormalization, SeparableConv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
)
from tensorflow.keras import backend as K

from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
# from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
# from keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
import os

# folder path
dir_path = r'/content/drive/MyDrive/data/data/Tomato Healthy'
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

In [ ]:
# Split data  to train ,  validation and test
# ! pip install split_folders
# import splitfolders 
# Home = '/content/drive/MyDrive/data/data'
# Aftersplit='/content/drive/MyDrive/split_data'
# splitfolders.ratio(Home, Aftersplit, seed=1337, ratio=(.8, 0.2))

In [ ]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 60
default_image_size = tuple((256, 256))
image_size = 0
directory_root = '/content/drive/MyDrive/data'
width=256
height=256
depth=3

In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:600]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

In [ ]:
image_size = len(image_list)
image_size

In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

In [ ]:
print(label_binarizer.classes_)

In [ ]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

In [ ]:
print("[INFO] Spliting data to train, test")
x_train, x_data, y_train, y_data = train_test_split(np_image_list, image_labels, test_size=0.3, random_state = 42) 
x_val, x_test, y_val, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state = 42) 

In [ ]:
len(x_val)

In [ ]:
aug = ImageDataGenerator(
    rotation_range=25, # rotation      
        zoom_range=[0.5,1], # zoom
				shear_range=.2 ,
        horizontal_flip=True, # horizontal flip
				fill_mode='nearest')

In [ ]:
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D

In [ ]:
model = Sequential()   
inputShape = (height, width, depth)
chanDim = -1  
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same", name="Conv2D"   ,  strides=(2,2)  ,   input_shape=inputShape))  # here  CL first layer in CNN
model.add(Activation("relu")) # here
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3),    name="MaxPooling2D")   )   # here   maMaxPooling    2 layer in CNN 
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3) ,  strides=(2,2), name="Conv2D1"  ))  # here
model.add(Activation("relu")) # here
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding ="same"  ,  strides=(2,2), name="Conv2D3"   )) # here
model.add(Activation("relu")) # here
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)  , name="MaxPooling2D2" ) ) # here
model.add(Dropout(0.25))  # To avoid the overfit problem by hide .25 from nuron in network 
model.add(Conv2D(128, (3, 3), padding="same" ,  strides=(2,2), name="Conv2D4"   ))  # here
model.add(Activation("relu"))  # here
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"   ,   strides=(2,2), name="Conv2D5"   )) # here
model.add(Activation("relu")) # here
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)  ,    name="MaxPooling2D3"))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))  # 1024 nuron of all network 
model.add(Activation("relu"))  # here  Activation Function 
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))    # Last layer for  Classification Depend on Number of class  8 classes
model.add(Activation("softmax")) #  softmax  for Multi Classification  , Sigmond For Binary Classification 
early_stopping = EarlyStopping(monitor='val_loss', patience=20, mode="auto")

In [ ]:
model.summary()

In [ ]:
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network
print("[INFO] training network...")

In [ ]:
history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_val, y_val),
    steps_per_epoch=len(x_train) // BS, 
    epochs=46, verbose=1 ,  callbacks=[early_stopping] )

In [ ]:
model.predict(x_test)

In [ ]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'ro', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'co', label='validation  accuracy')
plt.title('Training and validation  accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'yo', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='validation  loss')
plt.title('Training and validation  loss')
plt.legend()
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and Test accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Test accurarcy')
plt.title('Training and Test accurarcy')
plt.legend()

plt.figure()
#Train and Test loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Test loss')
plt.title('Training and Test loss')
plt.legend()
plt.show()

In [ ]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.ylabel('accuracy') 
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('loss') 
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
pred_train = model.predict(x_train)
pred_test = model.predict(x_test)  

In [ ]:
from sklearn.metrics import accuracy_score
print('Train Accuracy = ',accuracy_score(y_train,pred_train.round()))
print('Test Accuracy = ',accuracy_score(y_test,pred_test.round()))

In [ ]:
pred_test=np.argmax(pred_test, axis=1)
y_test=np.argmax(y_test, axis=1)
cm = confusion_matrix(y_test, pred_test)
print(cm)

In [ ]:
predicted_classes = model.predict(x_test)
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)
predicted_classes.shape, y_test.shape


In [ ]:
correct = np.where(predicted_classes==y_test)[0]
print("Found %d correct labels" % len(correct))
for i, correct in enumerate(correct[:8]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], y_test[correct]))
    plt.tight_layout()

In [ ]:
from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(n_classes)]
print(classification_report(y_test, predicted_classes, target_names=target_names))

In [ ]:
import seaborn as sns

fig, ax = plt.subplots(figsize=(20, 20))

ax = sns.heatmap(cm, annot=True, cmap='winter' , linewidths=.9)

ax.set_title('The Heatmap\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['Cabbage  mildew' , 'Cabbage Healthy' ,'Cucumber Healthy',
 ' Cucumber  mildew ' ,'Blight', 'Tomato Healthy',
 'Leaf miners' ,'lettuce healthy'])
ax.yaxis.set_ticklabels(['Cabbage  mildew' ,  'Cabbage Healthy' ,'Cucumber Healthy',
 ' Cucumber  mildew ' ,'Blight', 'Tomato Healthy',
 'Leaf miners' ,'lettuce healthy'])

## Display the visualization of the Confusion Matrix.
plt.show()




In [ ]:
#group_names = ['True Neg','False Pos','False Neg','True Pos','True Pos','True Pos','True Pos','True Pos','True Pos']
fig, ax = plt.subplots(figsize=(20, 20))
group_counts = ["{0:0.0f}".format(value) for value in
                cm.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cm.flatten()/np.sum(cm)]
labels = [f"{v1}\n{v2}\n" for v1, v2 in
          zip(group_counts,group_percentages)]
labels = np.asarray(labels).reshape(8,8)
ax = sns.heatmap(cm, annot=labels, fmt='', cmap='Blues')
ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Flower Category')
ax.set_ylabel('Actual Flower Category ');
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['Cabbage   diseases' , 'Cabbage Healthy' ,'Cucumber Healthy',
 'Cucumber diseases' ,'Tomato  diseases', 'Tomato Healthy',
 'lettuce  diseases' ,'lettuce healthy'])
ax.yaxis.set_ticklabels(['Cabbage   diseases' , 'Cabbage Healthy' ,'Cucumber Healthy',
 'Cucumber diseases' ,'Tomato  diseases', 'Tomato Healthy',
 'lettuce  diseases' ,'lettuce healthy'])
## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))

ax = sns.heatmap(cm/np.sum(cm), annot=True, 
            fmt='.2%', cmap='RdPu' , linewidths=.3)

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['Cabbage   diseases' , 'Cabbage Healthy' ,'Cucumber Healthy',
 'Cucumber diseases' ,'Tomato  diseases', 'Tomato Healthy',
 'lettuce  diseases' ,'lettuce healthy'])
ax.yaxis.set_ticklabels(['Cabbage   diseases' , 'Cabbage Healthy' ,'Cucumber Healthy',
 'Cucumber diseases' ,'Tomato  diseases', 'Tomato Healthy',
 'lettuce  diseases' ,'lettuce healthy'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
# save the model to disk
print("[INFO] Saving model...")
pickle.dump(model,open('cnn_model.pkl', 'wb'))